## Identifying and resolving conflicts in a batch of VI files

Andes Reinspection 
LRG
Data information can be found at https://data.desi.lbl.gov/desi/spectro/prospect/target-reinspection-andes/

/global/cfs/cdirs/desi/spectro/redux/andes


448 visual inspections of a total of 150 unique objects

32 conflicts

In [151]:
save_filename='truth_table_Andes_reinspection_LRG_68001_20200315_1.csv'

In [152]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

import VI_utils as VI
sys.path.append("/global/homes/r/tlan/prospect/prospect/py")

from prospect import utils_specviewer,plotframes
#import desispec
# library location will change ..
sys.path.append("/global/u2/t/tlan/python/prospect/py")
from prospect import utils_specviewer,plotframes
import matplotlib.pyplot as plt 

In [153]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/andes/tiles/'
tiles = ['68001']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']
#petals = ['0','3','6','7']
obs_db = utils_specviewer.make_targetdict(tiledir, petals=petals, tiles=tiles) # tiles = optional 
#del obs_db['67230-20200314']
#obs_db.keys()

In [154]:
pd.set_option('display.max_rows', 10)

In [155]:
#set to directory with all the VI files to merge
VI_dir = os.environ['HOME']+'/projects/VI_files/Andes_reinspection/LRG/'

In [156]:
#read a set of files 
all_files = os.listdir(VI_dir)
vi_files=[]

for i in range(1,4):
    pattern = "desi*_"+str(68001)+"_"+str(i)+"_*.csv"
    print(pattern)
    for entry in all_files:
        if fnmatch.fnmatch(entry, pattern):
            vi_files.append(entry)

print(vi_files)

desi*_68001_1_*.csv
desi*_68001_2_*.csv
desi*_68001_3_*.csv
['desi-vi_LRG_reinspection_68001_1_KSD.csv', 'desi-vi_LRG_reinspection_68001_1_HSH.csv', 'desi-vi_LRG_reinspection_68001_1_PNT.csv', 'desi-vi_LRG_reinspection_68001_2_PNT.csv', 'desi-vi_LRG_reinspection_68001_2_HSH.csv', 'desi-vi_LRG_reinspection_68001_2_KSD.csv', 'desi-vi_LRG_reinspection_68001_3_PNT.csv', 'desi-vi_LRG_reinspection_68001_3_KSD.csv']


In [157]:
vi = pd.read_csv(VI_dir + vi_files[0], delimiter = ",", engine='python')
print(vi_files[0])
for i in range(1,len(vi_files)):
    print(vi_files[i])
    vi2 = pd.read_csv(VI_dir + vi_files[i], delimiter = ",", engine='python')
    vi = vi.append(vi2, ignore_index=True)
#print(vi)    
#make groups of visual inspections, grouped by unique objects, and state number of single and multiple VIs
vi_gp = vi.groupby(['TARGETID'])

print('There are ' + str(len(vi)) + ' visual inspections of a total of ' + str(len(vi_gp)) + ' unique objects')
print('That gives ' + str(len(vi)/len(vi_gp)) + ' inspections per object.')

desi-vi_LRG_reinspection_68001_1_KSD.csv
desi-vi_LRG_reinspection_68001_1_HSH.csv
desi-vi_LRG_reinspection_68001_1_PNT.csv
desi-vi_LRG_reinspection_68001_2_PNT.csv
desi-vi_LRG_reinspection_68001_2_HSH.csv
desi-vi_LRG_reinspection_68001_2_KSD.csv
desi-vi_LRG_reinspection_68001_3_PNT.csv
desi-vi_LRG_reinspection_68001_3_KSD.csv
There are 399 visual inspections of a total of 150 unique objects
That gives 2.66 inspections per object.


In [158]:
#vi is a dataframe
#display(vi)

In [159]:
#vi.keys()
len(vi)

399

# Merge with zbest files
#Add: fiberID, delta_chi2, flux information,.. anything else?


In [160]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')

for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

#tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_panda()    
EXPID = list(set(tf['EXPID']))[0]
tf = tf[tf['EXPID']==EXPID]
tspec_df = tspec['TARGETID','DELTACHI2' ].to_pandas()
tf_df = tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_pandas()
print(len(tspec_df))
print(len(tf_df))
#tf_df = tf_df.rename(columns={"TargetID": "TARGETID"})
#tspec_df = tspec_df.rename(columns={"TargetID": "TARGETID"})
#print(len(vi))
vi = vi.merge(tf_df, how='left', on='TARGETID')
vi = vi.merge(tspec_df, how='left', on='TARGETID')
print(len(vi))

5000
5000
399


In [161]:
print(vi.keys())
#print(len(vi))
#vi['best redshift']=vi['VI_z']

Index(['TARGETID', 'EXPID', 'NIGHT', 'TILEID', 'Spec_version',
       'Redrock_version', 'Template_version', 'Redrock_spectype', 'Redrock_z',
       'VI_scanner', 'VI_quality', 'VI_issue', 'VI_z', 'VI_spectype',
       'VI_comment', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G',
       'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2'],
      dtype='object')


### Adding a bunch of useful columns

In [162]:
#vi['VI_z']
import VI_utils as VI
#vi['VI_z']
#bad_str = [s is None for s in vi['VI_comment']]
#vi['VI_comment'].isnull()
#bad_str = [s is None for s in vi['VI_comment']]
#print(bad_str)
#vi.loc[vi['VI_comment'].isnull(),'VI_comment']='--'
#vi['VI_comment']
#vi['all VI comments'] = vi.groupby('TARGETID')['VI_comment'].transform(lambda x: '|'.join(x))
#print(vi['all VI comments'])
vi = VI.prep_new_columns(vi)
#vi['best redshift'] = vi['VI_z']
#test = vi['VI_z']
#vi['best redshift'].isnull()
#vi.loc[vi['best redshift'].isnull(), 'best redshift'] = vi.loc[vi['best redshift'].isnull(), 'Redrock_z']
#vi['best redshift']

In [163]:
#check all the new columns (keys) have been added correctly
display(vi)
#print(vi.keys())

TARGETID  EXPID     NIGHT  TILEID Spec_version  Redrock_version  \
0    35191248222424420  55611  20200315   68001       0.34.4               -1   
1    35191248226618066  55611  20200315   68001       0.34.4               -1   
2    35191248226618068  55611  20200315   68001       0.34.4               -1   
3    35191248230812761  55611  20200315   68001       0.34.4               -1   
4    35191248230812876  55611  20200315   68001       0.34.4               -1   
..                 ...    ...       ...     ...          ...              ...   
394  35191259404437527  55611  20200315   68001       0.34.4               -1   
395  35191259404437886  55611  20200315   68001       0.34.4               -1   
396  35191259404438620  55611  20200315   68001       0.34.4               -1   
397  35191259404438871  55611  20200315   68001       0.34.4               -1   
398  35191259408632589  55611  20200315   68001       0.34.4               -1   

     Template_version Redrock_spectype  Redrock_z VI_scanner  ...   DELTACHI2  \
0                  -1           GALAXY     0.6752        KSD  ...   25.896473   
1                  -1           GALAXY     0.9883        KSD  ...   52.546031   
2                  -1           GALAXY     1.2007        KSD  ...   63.144346   
3                  -1           GALAXY     0.5667        KSD  ...   13.380124   
4                  -1           GALAXY     0.8970        KSD  ...   32.105794   
..                ...              ...        ...        ...  ...         ...   
394                -1           GALAXY     0.9219        KSD  ...  110.305561   
395                -1           GALAXY     0.8212        KSD  ...  169.067552   
396                -1           GALAXY     1.0256        KSD  ...  197.548935   
397                -1           GALAXY     0.8003        KSD  ...  108.110852   
398                -1           GALAXY     0.8969        KSD  ...   52.469745   

    best redshift  best spectype vi_combined_flag vi_diff   dz  vi_combined_z  \
0          0.6752         GALAXY         2.000000       0  0.0         0.6752   
1          0.9883         GALAXY         2.333333       1  0.0         0.9883   
2          1.2007         GALAXY         3.000000       0  0.0         1.2007   
3          0.5667         GALAXY         1.333333       1  0.0         0.5667   
4          0.8970         GALAXY         2.333333       1  0.0         0.8970   
..            ...            ...              ...     ...  ...            ...   
394        0.9219         GALAXY         3.500000       1  0.0         0.9219   
395        0.8212         GALAXY         3.500000       1  0.0         0.8212   
396        1.0256         GALAXY         4.000000       0  0.0         1.0256   
397        0.8003         GALAXY         4.000000       0  0.0         0.8003   
398        0.8969         GALAXY         2.500000       1  0.0         0.8969   

                                       all VI comments  N_VI  merger comment  
0    --|--|Continuum is shown; but no secure line d...     3            none  
1    --|--|Continuum detected OII doublet detected ...     3            none  
2                                             --|--|--     3            none  
3            --|--|Continuum detected No line detected     3            none  
4    --|--|Continuum detected OII doublet detected ...     3            none  
..                                                 ...   ...             ...  
394                                              --|--     2            none  
395  Interesting Although the Ca(H) and Ca(k) absor...     2            none  
396                                              --|--     2            none  
397  Bad spectrum at wavelength at wavelengths<6000...     2            none  
398                                              --|--     2            none  

[399 rows x 32 columns]

Get a table that holds only the objects that have been inspected more than once, and for which the individual VI classifications differ by 2 or more, or delta z / (1 + z) > 0.0033, or there is disagreement in best spectype (these are the conflicts to resolve)

In [164]:
vi_gp = vi.groupby(['TARGETID'])
vi_conflict = VI.find_conflicts(vi_gp)

In [165]:
len(vi_gp)

150

Get the target IDs of the problematic objects and display in table form for a quick summary:

In [166]:
unique_targets = np.unique(vi_conflict['TARGETID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35191251955353054 35191251963743475 35191251967938023 35191251972129169
 35191251976324782 35191255675700712 35191255679896938 35191255684089684
 35191255684090530 35191255688283211 35191255688284772 35191255692478016
 35191259370882446 35191259375076382 35191259387659595 35191259387661716
 35191263066063063 35191263070258456]
Total number of conflicts to resolve:  18


In [167]:
unique_targets[0]

35191251955353054

## This is where I resolve things manually - with care!!
### I think it's better to keep it in a notebook, as typos can be backtracked rather than a single manual edit of a text file

We edit either 'VI class', or 'best redshift' to resolve conflict. At the end, we look for conflicts again and we should find none.


In [168]:
#function to display the conflict in table format and open a prospect window
def display_conflict(conflict_id, show_spectra=None):
    
    #first, remind myself of the problem:
    display(vi[vi.TARGETID==unique_targets[conflict_id]][['TARGETID', 
       'Redrock_spectype', 'Redrock_z', 'VI_scanner', 'VI_quality', 'VI_issue',
       'VI_z', 'VI_spectype', 'VI_comment',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff',
       'vi_combined_z', 'dz', 'all VI comments', 'N_VI', 'merger comment']])

    spectra, zcat= utils_specviewer.load_spectra_zcat_from_targets([unique_targets[conflict_id]], tiledir, obs_db)
    # VI interface in notebook
    if show_spectra:
        plotframes.plotspectra(spectra, zcatalog=zcat, title='Target_select', notebook=True, mask_type='CMX_TARGET',with_vi_widgets=False)

In [169]:
#first, keep a safe copy of the original dataframe
vi_safe = vi.copy()

We will inspect each conflict on a prospect window, and resolve each conflict in turn

In [170]:
show_spectra = False

In [171]:
#Keep track of the conflicts by conflict_id

#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id, show_spectra=show_spectra)

TARGETID Redrock_spectype  Redrock_z VI_scanner  VI_quality  \
28   35191251955353054           GALAXY     0.6496        KSD           4   
78   35191251955353054           GALAXY     0.6496        HSH           2   
127  35191251955353054           GALAXY     0.6496        PNT           4   

    VI_issue    VI_z VI_spectype VI_comment  best redshift best spectype  \
28       NaN  0.6496      GALAXY         --         0.6496        GALAXY   
78       NaN     NaN         NaN         --         0.6496        GALAXY   
127      NaN  0.6496      GALAXY         --         0.6496        GALAXY   

     vi_combined_flag  vi_diff  vi_combined_z   dz all VI comments  N_VI  \
28           3.333333        2         0.6496  0.0        --|--|--     3   
78           3.333333        2         0.6496  0.0        --|--|--     3   
127          3.333333        2         0.6496  0.0        --|--|--     3   

    merger comment  
28            none  
78            none  
127           none

In [172]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.0885
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in quality. Redshif is robust.'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)

28     4
78     4
127    4
Name: VI_quality, dtype: int64
conflict ID =  0


In [173]:
#copy this text to a new cell to display the conflict
conflict_id=1
display_conflict(conflict_id, show_spectra=show_spectra)

TARGETID Redrock_spectype  Redrock_z VI_scanner  VI_quality  \
9    35191251963743475           GALAXY     0.8101        KSD           3   
59   35191251963743475           GALAXY     0.8101        HSH           3   
109  35191251963743475           GALAXY     0.8101        PNT           3   

    VI_issue    VI_z VI_spectype  \
9        NaN  0.8201      GALAXY   
59       NaN     NaN         NaN   
109      NaN  0.8201      GALAXY   

                                            VI_comment  best redshift  \
9                                                   --         0.8201   
59                                                  --         0.8101   
109  OII doublet detected but there is an additiona...         0.8201   

    best spectype  vi_combined_flag  vi_diff  vi_combined_z        dz  \
9          GALAXY               3.0        0         0.8101  0.005525   
59         GALAXY               3.0        0         0.8101  0.005525   
109        GALAXY               3.0        0         0.8101  0.005525   

                                       all VI comments  N_VI merger comment  
9    --|--|OII doublet detected but there is an add...     3           none  
59   --|--|OII doublet detected but there is an add...     3           none  
109  --|--|OII doublet detected but there is an add...     3           none

In [174]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.8101
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] =3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Redshift disagreement. '
##look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
#print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)



conflict ID =  1


In [175]:
#copy this text to a new cell to display the conflict
conflict_id=2
display_conflict(conflict_id, show_spectra=show_spectra)

TARGETID Redrock_spectype  Redrock_z VI_scanner  VI_quality  \
17   35191251967938023           GALAXY     0.7626        KSD           2   
67   35191251967938023           GALAXY     0.7626        HSH           2   
116  35191251967938023           GALAXY     0.7626        PNT           4   

    VI_issue    VI_z VI_spectype VI_comment  best redshift best spectype  \
17       NaN  0.7626      GALAXY         --         0.7626        GALAXY   
67       NaN     NaN         NaN         --         0.7626        GALAXY   
116      NaN  0.7626      GALAXY         --         0.7626        GALAXY   

     vi_combined_flag  vi_diff  vi_combined_z   dz all VI comments  N_VI  \
17           2.666667        2         0.7626  0.0        --|--|--     3   
67           2.666667        2         0.7626  0.0        --|--|--     3   
116          2.666667        2         0.7626  0.0        --|--|--     3   

    merger comment  
17            none  
67            none  
116           none

In [176]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.2320
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in quality. Low S/N CaII. Be conservative'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  2


In [177]:
#copy this text to a new cell to display the conflict
conflict_id=3
display_conflict(conflict_id, show_spectra=show_spectra)


TARGETID Redrock_spectype  Redrock_z VI_scanner  VI_quality  \
19   35191251972129169           GALAXY     0.9099        KSD           1   
69   35191251972129169           GALAXY     0.9099        HSH           3   
118  35191251972129169           GALAXY     0.9099        PNT           2   

    VI_issue    VI_z VI_spectype  \
19       NaN  0.9099      GALAXY   
69       NaN     NaN         NaN   
118      NaN  0.9099      GALAXY   

                                            VI_comment  best redshift  \
19                                                  --         0.9099   
69                                                  --         0.9099   
118  Continuum detected low SNR for absorptions No ...         0.9099   

    best spectype  vi_combined_flag  vi_diff  vi_combined_z   dz  \
19         GALAXY               2.0        2         0.9099  0.0   
69         GALAXY               2.0        2         0.9099  0.0   
118        GALAXY               2.0        2         0.9099  0.0   

                                       all VI comments  N_VI merger comment  
19   --|--|Continuum detected low SNR for absorptio...     3           none  
69   --|--|Continuum detected low SNR for absorptio...     3           none  
118  --|--|Continuum detected low SNR for absorptio...     3           none

In [178]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.5128
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY' 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in quality. CaII in bad spectral region. Be conservative.'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
#print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)


conflict ID =  3


In [179]:
#copy this text to a new cell to display the conflict
conflict_id=4
display_conflict(conflict_id, show_spectra=show_spectra)


TARGETID Redrock_spectype  Redrock_z VI_scanner  VI_quality  \
44   35191251976324782           GALAXY     0.5559        KSD           4   
94   35191251976324782           GALAXY     0.5559        HSH           4   
143  35191251976324782           GALAXY     0.5559        PNT           2   

    VI_issue    VI_z VI_spectype VI_comment  best redshift best spectype  \
44       NaN  0.5559      GALAXY         --         0.5559        GALAXY   
94       NaN     NaN         NaN         --         0.5559        GALAXY   
143      NaN  0.5559      GALAXY         --         0.5559        GALAXY   

     vi_combined_flag  vi_diff  vi_combined_z   dz all VI comments  N_VI  \
44           3.333333        2         0.5559  0.0        --|--|--     3   
94           3.333333        2         0.5559  0.0        --|--|--     3   
143          3.333333        2         0.5559  0.0        --|--|--     3   

    merger comment  
44            none  
94            none  
143           none

In [180]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.3210
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. '
#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
#print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)

conflict ID =  4


In [181]:
#copy this text to a new cell to display the conflict
conflict_id=5
display_conflict(conflict_id, show_spectra=show_spectra)

TARGETID Redrock_spectype  Redrock_z VI_scanner  VI_quality  \
36   35191255675700712           GALAXY     0.7605        KSD           4   
86   35191255675700712           GALAXY     0.7605        HSH           4   
135  35191255675700712           GALAXY     0.7605        PNT          -1   

    VI_issue    VI_z VI_spectype                    VI_comment  best redshift  \
36       NaN  0.7605      GALAXY                            --         0.7605   
86       NaN     NaN         NaN                            --         0.7605   
135      NaN     NaN      GALAXY  Hbeta detected as absorption         0.7605   

    best spectype  vi_combined_flag  vi_diff  vi_combined_z   dz  \
36         GALAXY          2.333333        5         0.7605  0.0   
86         GALAXY          2.333333        5         0.7605  0.0   
135        GALAXY          2.333333        5         0.7605  0.0   

                        all VI comments  N_VI merger comment  
36   --|--|Hbeta detected as absorption     3           none  
86   --|--|Hbeta detected as absorption     3           none  
135  --|--|Hbeta detected as absorption     3           none

In [182]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Redshift is robust.'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  5


In [183]:
#copy this text to a new cell to display the conflict
conflict_id=6
display_conflict(conflict_id, show_spectra=show_spectra)


TARGETID Redrock_spectype  Redrock_z VI_scanner  VI_quality  \
43   35191255679896938           GALAXY     0.5131        KSD           4   
93   35191255679896938           GALAXY     0.5131        HSH           2   
142  35191255679896938           GALAXY     0.5131        PNT           3   

    VI_issue    VI_z VI_spectype VI_comment  best redshift best spectype  \
43       NaN  0.0031        STAR         --         0.0031          STAR   
93       NaN     NaN         NaN         --         0.5131        GALAXY   
142      NaN  0.5131      GALAXY         --         0.5131        GALAXY   

     vi_combined_flag  vi_diff  vi_combined_z        dz all VI comments  N_VI  \
43                3.0        2         0.5131  0.508424        --|--|--     3   
93                3.0        2         0.5131  0.508424        --|--|--     3   
142               3.0        2         0.5131  0.508424        --|--|--     3   

    merger comment  
43            none  
93            none  
142           none

In [184]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 1.5
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = ' A blended object of galaxy+star. Low VI quality to remove it from the catalog. '

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  6


In [185]:
#copy this text to a new cell to display the conflict
conflict_id=7
display_conflict(conflict_id, show_spectra=show_spectra)

TARGETID Redrock_spectype  Redrock_z VI_scanner  VI_quality  \
150  35191255684089684           GALAXY     0.8315        PNT           2   
200  35191255684089684           GALAXY     0.8315        HSH           4   
250  35191255684089684           GALAXY     0.8315        KSD           4   

    VI_issue    VI_z VI_spectype VI_comment  best redshift best spectype  \
150        S  0.8315      GALAXY         --         0.8315        GALAXY   
200      NaN     NaN         NaN         --         0.8315        GALAXY   
250      NaN  0.8315         NaN         --         0.8315        GALAXY   

     vi_combined_flag  vi_diff  vi_combined_z   dz all VI comments  N_VI  \
150          3.333333        2         0.8315  0.0        --|--|--     3   
200          3.333333        2         0.8315  0.0        --|--|--     3   
250          3.333333        2         0.8315  0.0        --|--|--     3   

    merger comment  
150           none  
200           none  
250           none

In [186]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.1847
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Reshift is secure'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  7


In [187]:
#copy this text to a new cell to display the conflict
conflict_id=8
display_conflict(conflict_id, show_spectra=show_spectra)


TARGETID Redrock_spectype  Redrock_z VI_scanner  VI_quality  \
152  35191255684090530           GALAXY     0.6725        PNT           2   
202  35191255684090530           GALAXY     0.6725        HSH           2   
252  35191255684090530           GALAXY     0.6725        KSD           4   

    VI_issue    VI_z VI_spectype               VI_comment  best redshift  \
152      NaN     NaN      GALAXY  Low SNR possibly an LRG         0.6725   
202      NaN     NaN         NaN                       --         0.6725   
252      NaN  0.6725         NaN                       --         0.6725   

    best spectype  vi_combined_flag  vi_diff  vi_combined_z   dz  \
152        GALAXY          2.666667        2         0.6725  0.0   
202        GALAXY          2.666667        2         0.6725  0.0   
252        GALAXY          2.666667        2         0.6725  0.0   

                   all VI comments  N_VI merger comment  
152  Low SNR possibly an LRG|--|--     3           none  
202  Low SNR possibly an LRG|--|--     3           none  
252  Low SNR possibly an LRG|--|--     3           none

In [188]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.1920
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = "Redshift is secure with a bit low S/N."

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  8


In [189]:
#copy this text to a new cell to display the conflict
conflict_id=9
display_conflict(conflict_id, show_spectra=show_spectra)


TARGETID Redrock_spectype  Redrock_z VI_scanner  VI_quality  \
155  35191255688283211           GALAXY     1.6854        PNT           1   
205  35191255688283211           GALAXY     1.6854        HSH           1   
255  35191255688283211           GALAXY     1.6854        KSD           3   

    VI_issue    VI_z VI_spectype  \
155        R  1.6854      GALAXY   
205      NaN     NaN         NaN   
255      NaN  0.7966         NaN   

                                            VI_comment  best redshift  \
155  Continuum detected but no feature is detected ...         1.6854   
205                                                 --         1.6854   
255                            Two objects in spectrum         0.7966   

    best spectype  vi_combined_flag  vi_diff  vi_combined_z        dz  \
155        GALAXY          1.666667        2         1.6854  0.494712   
205        GALAXY          1.666667        2         1.6854  0.494712   
255        GALAXY          1.666667        2         1.6854  0.494712   

                                       all VI comments  N_VI merger comment  
155  Continuum detected but no feature is detected ...     3           none  
205  Continuum detected but no feature is detected ...     3           none  
255  Continuum detected but no feature is detected ...     3           none

In [190]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.7966
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Redshift is not secure.'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  9


In [191]:
#copy this text to a new cell to display the conflict
conflict_id=10
display_conflict(conflict_id, show_spectra=show_spectra)

TARGETID Redrock_spectype  Redrock_z VI_scanner  VI_quality  \
160  35191255688284772           GALAXY     0.8924        PNT           2   
210  35191255688284772           GALAXY     0.8924        HSH           3   
260  35191255688284772           GALAXY     0.8924        KSD           4   

    VI_issue    VI_z VI_spectype  \
160      NaN  0.8924      GALAXY   
210      NaN     NaN         NaN   
260      NaN  0.8924         NaN   

                                            VI_comment  best redshift  \
160  Continuum deteced  but features are detected w...         0.8924   
210                                                 --         0.8924   
260                                                 --         0.8924   

    best spectype  vi_combined_flag  vi_diff  vi_combined_z   dz  \
160        GALAXY               3.0        2         0.8924  0.0   
210        GALAXY               3.0        2         0.8924  0.0   
260        GALAXY               3.0        2         0.8924  0.0   

                                       all VI comments  N_VI merger comment  
160  Continuum deteced  but features are detected w...     3           none  
210  Continuum deteced  but features are detected w...     3           none  
260  Continuum deteced  but features are detected w...     3           none

In [192]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 1.6283
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. Redshift is robust'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  10


In [193]:
#copy this text to a new cell to display the conflict
conflict_id=11
display_conflict(conflict_id, show_spectra=show_spectra)

TARGETID Redrock_spectype  Redrock_z VI_scanner  VI_quality  \
167  35191255692478016           GALAXY     1.6844        PNT           2   
217  35191255692478016           GALAXY     1.6844        HSH           1   
267  35191255692478016           GALAXY     1.6844        KSD           3   

    VI_issue    VI_z VI_spectype  \
167      NaN  1.6844      GALAXY   
217      NaN     NaN         NaN   
267      NaN  1.1877         NaN   

                                            VI_comment  best redshift  \
167  Continuum detected But no feature detected Pos...         1.6844   
217                                                 --         1.6844   
267                                                 --         1.1877   

    best spectype  vi_combined_flag  vi_diff  vi_combined_z        dz  \
167        GALAXY               2.0        2         1.6844  0.227042   
217        GALAXY               2.0        2         1.6844  0.227042   
267        GALAXY               2.0        2         1.6844  0.227042   

                                       all VI comments  N_VI merger comment  
167  Continuum detected But no feature detected Pos...     3           none  
217  Continuum detected But no feature detected Pos...     3           none  
267  Continuum detected But no feature detected Pos...     3           none

In [194]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 1.1877
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'CaII at 1.1877'
#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  11


In [195]:
#copy this text to a new cell to display the conflict
conflict_id=12
display_conflict(conflict_id, show_spectra=show_spectra)



TARGETID Redrock_spectype  Redrock_z VI_scanner  VI_quality  \
172  35191259370882446           GALAXY     0.1976        PNT           1   
222  35191259370882446           GALAXY     0.1976        HSH           1   
272  35191259370882446           GALAXY     0.1976        KSD           4   

    VI_issue    VI_z VI_spectype  \
172       RS     NaN      GALAXY   
222      NaN     NaN         NaN   
272        S  0.8376         NaN   

                                            VI_comment  best redshift  \
172  Continuum detected But no feature is detected ...         0.1976   
222                                                 --         0.1976   
272                significant residual around 4800 AA         0.8376   

    best spectype  vi_combined_flag  vi_diff  vi_combined_z        dz  \
172        GALAXY               2.0        3         0.1976  0.534402   
222        GALAXY               2.0        3         0.1976  0.534402   
272        GALAXY               2.0        3         0.1976  0.534402   

                                       all VI comments  N_VI merger comment  
172  Continuum detected But no feature is detected ...     3           none  
222  Continuum detected But no feature is detected ...     3           none  
272  Continuum detected But no feature is detected ...     3           none

In [196]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.8376
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. Redshift is secure. '

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  12


In [197]:
#copy this text to a new cell to display the conflict
conflict_id=13
display_conflict(conflict_id, show_spectra=show_spectra)


TARGETID Redrock_spectype  Redrock_z VI_scanner  VI_quality  \
177  35191259375076382           GALAXY     0.6475        PNT           4   
227  35191259375076382           GALAXY     0.6475        HSH           2   
277  35191259375076382           GALAXY     0.6475        KSD           2   

    VI_issue    VI_z VI_spectype VI_comment  best redshift best spectype  \
177      NaN  0.6475      GALAXY         --         0.6475        GALAXY   
227      NaN     NaN         NaN         --         0.6475        GALAXY   
277      NaN  0.6475         NaN         --         0.6475        GALAXY   

     vi_combined_flag  vi_diff  vi_combined_z   dz all VI comments  N_VI  \
177          2.666667        2         0.6475  0.0        --|--|--     3   
227          2.666667        2         0.6475  0.0        --|--|--     3   
277          2.666667        2         0.6475  0.0        --|--|--     3   

    merger comment  
177           none  
227           none  
277           none

In [198]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Low S/N. Be conservative.'
#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  13


In [199]:
#copy this text to a new cell to display the conflict
conflict_id=14
display_conflict(conflict_id, show_spectra=show_spectra)

TARGETID Redrock_spectype  Redrock_z VI_scanner  VI_quality  \
301  35191259387659595           GALAXY     0.9731        PNT           1   
351  35191259387659595           GALAXY     0.9731        KSD           3   

    VI_issue    VI_z VI_spectype  \
301      NaN  0.9731      GALAXY   
351      NaN  0.9731         NaN   

                                            VI_comment  best redshift  \
301  Interesting Continuum is resolved But Hdelta a...         0.9731   
351                                                 --         0.9731   

    best spectype  vi_combined_flag  vi_diff  vi_combined_z   dz  \
301        GALAXY               2.0        2         0.9731  0.0   
351        GALAXY               2.0        2         0.9731  0.0   

                                       all VI comments  N_VI merger comment  
301  Interesting Continuum is resolved But Hdelta a...     2           none  
351  Interesting Continuum is resolved But Hdelta a...     2           none

In [200]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Redshift is robust'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  14


In [201]:
#copy this text to a new cell to display the conflict
conflict_id=15
display_conflict(conflict_id, show_spectra=show_spectra)



TARGETID Redrock_spectype  Redrock_z VI_scanner  VI_quality  \
308  35191259387661716           GALAXY      0.549        PNT           2   
358  35191259387661716           GALAXY      0.549        KSD           4   

    VI_issue   VI_z VI_spectype                                VI_comment  \
308      NaN  0.549      GALAXY  Continuum detected D1 D2 resolved as one   
358      NaN  0.549         NaN                                        --   

     best redshift best spectype  vi_combined_flag  vi_diff  vi_combined_z  \
308          0.549        GALAXY               3.0        2          0.549   
358          0.549        GALAXY               3.0        2          0.549   

               dz                              all VI comments  N_VI  \
308  7.167353e-17  Continuum detected D1 D2 resolved as one|--     2   
358  7.167353e-17  Continuum detected D1 D2 resolved as one|--     2   

    merger comment  
308           none  
358           none

In [202]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 1.0465
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Redshift is roubst'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  15


In [203]:
#copy this text to a new cell to display the conflict
conflict_id=16
display_conflict(conflict_id, show_spectra=show_spectra)


TARGETID Redrock_spectype  Redrock_z VI_scanner  VI_quality  \
325  35191263066063063           GALAXY     0.9281        PNT           4   
375  35191263066063063           GALAXY     0.9281        KSD           2   

    VI_issue    VI_z VI_spectype VI_comment  best redshift best spectype  \
325      NaN  0.9281      GALAXY         --         0.9281        GALAXY   
375      NaN  0.9281         NaN         --         0.9281        GALAXY   

     vi_combined_flag  vi_diff  vi_combined_z   dz all VI comments  N_VI  \
325               3.0        2         0.9281  0.0           --|--     2   
375               3.0        2         0.9281  0.0           --|--     2   

    merger comment  
325           none  
375           none

In [204]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.9649
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, CaII in bad spectral region. Be conservative'
#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  16


In [205]:
#copy this text to a new cell to display the conflict

conflict_id=17
display_conflict(conflict_id, show_spectra=show_spectra)



TARGETID Redrock_spectype  Redrock_z VI_scanner  VI_quality  \
316  35191263070258456           GALAXY     0.9657        PNT           2   
366  35191263070258456           GALAXY     0.9657        KSD           4   

    VI_issue    VI_z VI_spectype  \
316      NaN  0.9657      GALAXY   
366      NaN  0.9657         NaN   

                                            VI_comment  best redshift  \
316  Hdelta and Hgamma detected as absorption lines...         0.9657   
366                                                 --         0.9657   

    best spectype  vi_combined_flag  vi_diff  vi_combined_z   dz  \
316        GALAXY               3.0        2         0.9657  0.0   
366        GALAXY               3.0        2         0.9657  0.0   

                                       all VI comments  N_VI merger comment  
316  Hdelta and Hgamma detected as absorption lines...     2           none  
366  Hdelta and Hgamma detected as absorption lines...     2           none

In [206]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 1.1799
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Redshift is secure.'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  17


### and so on...

We should now recompute the conflicts, and not find any.

In [207]:
vi_conflict = VI.find_conflicts(vi_gp)

In [208]:
unique_targets = np.unique(vi_conflict['TARGETID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


Display anything that was missed (if "Total number of conflicts" isn't zero) and resolve!

Let's check the number of conflicts again.

In [209]:
vi_conflict = VI.find_conflicts(vi_gp)

In [210]:
unique_targets = np.unique(vi_conflict['TARGETID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


## Woohoo!

## Finally
we need to recompute vi_combined_flag and vi_combined_z, after everything is resolved.

In [211]:
vi['vi_combined_flag'] = vi.groupby('TARGETID')['VI_quality'].transform('mean')
vi['vi_combined_z'] = vi.groupby('TARGETID')['best redshift'].transform('mean')


## Now we prepare to write to file. 

### The important columns for the truth table construction are **vi_combined_flag** and **vi_combined_z** 

The truth table should **take the redhift value in vi_combined_z** for all objects with **vi_combined_flag >= 2.5** (for discussion, but this catches the cases where there is some disagreement on the 2/3 boundary).

After the merging process, those two columns will be the same for ALL of the members of each group by TargetID, so it doesn't matter which member we write to file. Taking the 1st one just makes this easy. 


In [212]:
vi.keys()

Index(['TARGETID', 'EXPID', 'NIGHT', 'TILEID', 'Spec_version',
       'Redrock_version', 'Template_version', 'Redrock_spectype', 'Redrock_z',
       'VI_scanner', 'VI_quality', 'VI_issue', 'VI_z', 'VI_spectype',
       'VI_comment', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G',
       'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2', 'best redshift',
       'best spectype', 'vi_combined_flag', 'vi_diff', 'dz', 'vi_combined_z',
       'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')

In [213]:
print(VI_dir+save_filename)

/global/homes/t/tlan/projects/VI_files/Andes_reinspection/LRG/truth_table_Andes_reinspection_LRG_68001_20200315_1.csv


In [214]:
vi_gp['Redrock_z', 'Redrock_spectype', 'best redshift' , 'best spectype', 'vi_combined_flag',  
      'all VI comments', 'merger comment', 'N_VI', 'DELTACHI2','FIBER'].first().to_csv(VI_dir+save_filename)

<ipython-input-214-2063ed20beca>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  vi_gp['Redrock_z', 'Redrock_spectype', 'best redshift' , 'best spectype', 'vi_combined_flag',


Check that merged file reads in OK - check comments

In [215]:
merged_file = pd.read_csv(VI_dir+save_filename)

In [216]:
merged_file.keys()

Index(['TARGETID', 'Redrock_z', 'Redrock_spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'all VI comments',
       'merger comment', 'N_VI', 'DELTACHI2', 'FIBER'],
      dtype='object')

In [217]:
len(merged_file)

150

In [218]:
VI_dir

'/global/homes/t/tlan/projects/VI_files/Andes_reinspection/LRG/'